# 📘 Admission List Cleaning Notebook
This notebook walks through the steps used to clean and consolidate the admission list Excel file for analysis.

In [1]:
import pandas as pd
import numpy as np

## 📥 Load Excel File

In [2]:
file_path = 'Admission.xlsx'
df_raw = pd.read_excel(file_path, sheet_name='Table 1', header=None)
df_raw.head(20)

,0,1,2,3,4,5,6,7,8,9,10
0,MAKERERE UNIVERSITY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ACADEMIC REGISTRAR'S DEPARTMENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ADMISSION LISTS FOR 2025/2026 ACADEMIC YEAR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PRIVATE SPONSORSHIP- DIRECT ENTRY SCHEME,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BSC. Accounting (EVE),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,no,formid,index_no,name,ge,uace_,dcode,dname,course_code,course_name,tot_wt
6,1,25APF900350094290048,U2789/905,NIMUSIIMA SANDRAH,F,2024,55,WAKISO,ACC,BSC. Accounting (EVE),42.7
7,2,25APF561140686250786,U0025/529,BULUKUKU DANIEL EPHRAIM,M,2014,93,KIBUKU,ACC,BSC. Accounting (EVE),42.3
8,3,25APF530278631275455,U2977/518,MIREMBE ELIZABETH,F,2024,16,KAMPALA,ACC,BSC. Accounting (EVE),41.1
9,4,25APF1346250173272930,U1664/712,NAMATA MARTHA,F,2024,33,MASAKA,ACC,BSC. Accounting (EVE),40.7


## 🔍 Inspecting Raw Structure

- The file has introductory rows (unstructured).
- Each table has a row with headers like `'no', 'formid', ...`.
- Tables are grouped under program names (e.g. `BSC. Accounting (EVE)`).

## 🧹 Extract and Clean Each Table

In [3]:
tables = []
current_program = None

for i in range(len(df_raw)):
    row = df_raw.iloc[i]
    if str(row[0]).strip().lower() == 'no' and str(row[1]).strip().lower() == 'formid':
        headers = df_raw.iloc[i]
        data_start = i + 1
        data_end = data_start
        while data_end < len(df_raw) and not pd.isna(df_raw.iloc[data_end, 0]):
            data_end += 1

        table = df_raw.iloc[data_start:data_end].copy()
        table.columns = headers
        table = table.dropna(how='all')

        for j in range(i-1, -1, -1):
            candidate = str(df_raw.iloc[j, 0]).strip()
            if candidate and not candidate.lower().startswith('makerere') and 'admission' not in candidate.lower():
                current_program = candidate
                break

        table['program_name'] = current_program
        tables.append(table)

In [4]:
# Convert 'dcode' to a nullable integer type
df_combined['dcode'] = df_combined['dcode'].astype('Int64')

# Fill missing 'dcode' where 'dname' is 'LUWERO'
df_combined.loc[(df_combined['dname'].str.upper() == 'LUWERO') & (df_combined['dcode'].isna()), 'dcode'] = 32

# Check the rows with 'LUWERO' to confirm
df_combined[df_combined['dname'].str.upper() == 'LUWERO']


NameError: name 'df_combined' is not defined

## 🔗 Combine All Tables

In [ ]:
df_combined = pd.concat(tables, ignore_index=True)
df_combined.head()

,no,formid,index_no,name,ge,uace_,dcode,dname,course_code,course_name,tot_wt,program_name
0,1,25APF900350094290048,U2789/905,NIMUSIIMA SANDRAH,F,2024,55,WAKISO,ACC,BSC. Accounting (EVE),42.7,BSC. Accounting (EVE)
1,2,25APF561140686250786,U0025/529,BULUKUKU DANIEL EPHRAIM,M,2014,93,KIBUKU,ACC,BSC. Accounting (EVE),42.3,BSC. Accounting (EVE)
2,3,25APF530278631275455,U2977/518,MIREMBE ELIZABETH,F,2024,16,KAMPALA,ACC,BSC. Accounting (EVE),41.1,BSC. Accounting (EVE)
3,4,25APF1346250173272930,U1664/712,NAMATA MARTHA,F,2024,33,MASAKA,ACC,BSC. Accounting (EVE),40.7,BSC. Accounting (EVE)
4,5,25APF1553620758303031,U0004/754,NAKALEMA JOYCE,F,2024,33,MASAKA,ACC,BSC. Accounting (EVE),40.3,BSC. Accounting (EVE)


,no,formid,index_no,name,ge,uace_,dcode,dname,course_code,course_name,tot_wt,program_name
10379,42,25APF1096305957317713,ZZVX90YU,NAMUYANJA Masituula,F,2024,32,LUWERO,BSA,BSC. Accounting,39.2,BSC. Accounting(Day)


## Export Cleaned Data

In [ ]:
df_combined.to_excel('Cleaned_Admission_List.xlsx', index=False)